In [94]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
from statsmodels.graphics.tsaplots import plot_acf
from scipy import stats

# Read in the data
df = pd.read_csv('Datasets/dataset_mood_smartphone.csv')

In [103]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
from statsmodels.graphics.tsaplots import plot_acf
from scipy import stats


def group_df_and_aggregate(dataframe):
    dataframe['date'] = pd.to_datetime(dataframe['time']).dt.date
    dataframe['time'] = pd.to_datetime(dataframe['time']).dt.time

    grouped_df = dataframe.groupby(['id', 'date', 'time', 'variable']).sum().reset_index()

    #create dataframe of values per id per date per time
    dataframe = grouped_df.pivot_table(index=['id', 'date', 'time'], columns='variable', values='value')

    return dataframe

# Remove instances that are not in the range of the depicted column
# ATTENTION should be performed before merging the data!!
def range_removal(dataframe, lower, upper, column_name, keep_nan=True):

    column = dataframe[column_name]

    if keep_nan:
        # If we want to keep the NaN values use this
        filtered_df = dataframe[column.between(lower, upper) | column.isna()]
    else:
        # If we want to remove the NaN values use this
        filtered_df = dataframe[(column >= lower) & (column <= upper)]

    return filtered_df

# Remove the negative instances for the applications
def remove_negative_values(columns, dataframe):
    for column in columns:
        # Keep the NaN values and replace the negative values with 0
        dataframe[column] = dataframe.loc[~(dataframe[column] < 0), column]

    return dataframe

# TO DO: extreme values removal

# Apply different aggregation functions to different variables
def aggregation_for_variables(dataframe):
    agg_dict = {'mood': 'mean', 'circumplex.arousal': 'mean', 'circumplex.valence': 'mean', 'activity': 'mean', 'screen': 'sum', 'call': 'sum', 'sms': 'sum', 'appCat.builtin': 'sum', 'appCat.communication': 'sum', 'appCat.entertainment': 'sum', 'appCat.finance': 'sum', 'appCat.game': 'sum', 'appCat.office': 'sum', 'appCat.other': 'sum', 'appCat.social': 'sum', 'appCat.travel': 'sum', 'appCat.unknown': 'sum', 'appCat.weather': 'sum', 'appCat.utilities': 'sum'}

    # Create df where some of the columns values are summed and for some the mean is taken for each day
    dataframe = dataframe.groupby(['date', 'id']).agg(agg_dict).reset_index()
    return dataframe


# Function that removes the starting nan values
def remove_starting_nan_until_n_values(column_name, dataframe, n_param):
    '''
    params: 
        colmun_name -> takes the name of the column as a string
        dataframe -> takes the dataframe object
        n_params -> int that takes the number of real float values before the NaN are removed

    This function removes the starting NaN values before an amount of n float values are detected
    Therefore removing all the instances before any real measurements are performed
    '''

    verbose = False

    unique_users = dataframe['id'].unique()
    new_df = pd.DataFrame()

    for user_i in unique_users:
        df_for_user_i = dataframe[dataframe['id'] == user_i]
        count_nan = 0
        count_float = 0
        count_final = 0

        for value in df_for_user_i[column_name]:
            print('-'*50, '\n', value) if verbose else None
            if math.isnan(value):
                print('NAN', value, type(value), isinstance(value, float)) if verbose else None
                count_nan += 1
                count_float = 0
            if isinstance(value, float):
                print("FLOAT",value, type(value), isinstance(value, float)) if verbose else None
                count_float += 1
            if count_float > n_param:
                n = count_nan 
                added_df = df_for_user_i.iloc[n:]
                added_df
                new_df = new_df.append(added_df)
                break

    return new_df


# Create a new column that contains the mean of the previous and next value
# NOTE can still change to take mean of previous n and next n values
def mean_between(column_name, dataframe):

    # sort the dataframe by id
    dataframe = dataframe.sort_values(by='date')

    # get the previous and next NaN values
    dataframe['prev_value'] = dataframe.groupby('id')[column_name].apply(lambda x: x.shift(1).bfill())
    dataframe['next_value'] = dataframe.groupby('id')[column_name].apply(lambda x: x.shift(-1).ffill())

    # take the mean of previous and next values
    dataframe['mean_value'] = dataframe[['prev_value', 'next_value']].mean(axis=1)

    for index, row in dataframe.iterrows():
        if pd.isna(row[column_name]):
            dataframe.loc[index, column_name] = row['mean_value']

    dataframe.drop(['mean_value', 'prev_value', 'next_value'], axis=1, inplace=True)

    return dataframe


def MA_on_missing_values(dataframe, column, n):
    '''
    Performs moving averages on a column for each distinct 'id' in the 'mood' column,
    and creates a new column where NaN values are substituted
    '''
    # Group by 'id' column
    groups = dataframe.groupby('id')

    # Perform moving averages for each group
    dataframe['MA'] = groups[column].ewm(span=n).mean().reset_index(0, drop=True)

    for index, row in dataframe.iterrows():
        if pd.isna(row[column]):
            dataframe.loc[index, column] = row['MA']

    dataframe.drop('MA', axis=1, inplace=True)
    
    return dataframe


def select_imputation_technique(mean_values, dataframe):

    if mean_values == True:
        dataframe = mean_between('mood', dataframe)
        dataframe = mean_between('circumplex.arousal', dataframe)
        dataframe = mean_between('circumplex.valence', dataframe)
        dataframe = mean_between('activity', dataframe)
    else:
        dataframe = MA_on_missing_values(dataframe, 'mood', 10)
        dataframe = MA_on_missing_values(dataframe, 'circumplex.arousal', 10)
        dataframe = MA_on_missing_values(dataframe, 'circumplex.valence', 10)
        dataframe = MA_on_missing_values(dataframe, 'activity', 10)

    return dataframe


# HERE WE START THE PIPELINE

#every file should have a main that is called for the pipeline
def main(df):
    df = pd.read_csv('Datasets/dataset_mood_smartphone.csv')

    df = group_df_and_aggregate(df)

    df = range_removal(df,1, 10, 'mood')
    df = range_removal(df,-2, 2, 'circumplex.arousal')
    df = range_removal(df,-2, 2, 'circumplex.valence')
    df = range_removal(df, 0, 1, 'activity')

    df = remove_negative_values(['appCat.builtin', 'appCat.communication', 'appCat.entertainment', 'appCat.finance', 'appCat.game', 'appCat.office', 'appCat.other', 'appCat.social', 'appCat.travel', 'appCat.unknown', 'appCat.utilities', 'appCat.weather', 'screen', 'call', 'sms'], df)

    # TO DO: remove extreme values

    df = aggregation_for_variables(df)

    # Need to run two times (sorry voor buggy code)
    df = remove_starting_nan_until_n_values('mood', df, 4)
    df = remove_starting_nan_until_n_values('mood', df, 4)

    df = select_imputation_technique(True, df)

    df.to_csv('Datasets/cleaned_data.csv', index=False)
    return df








In [95]:
df['date'] = pd.to_datetime(df['time']).dt.date
df['time'] = pd.to_datetime(df['time']).dt.time

grouped_df = df.groupby(['id', 'date', 'time', 'variable']).sum().reset_index()

#create dataframe of values per id per date per time
df = grouped_df.pivot_table(index=['id', 'date', 'time'], columns='variable', values='value')

In [96]:
# Remove instances that are not in the range of the depicted column
# ATTENTION should be performed before merging the data!!
def range_removal(lower, upper, column_name, keep_nan=True):

    column = df[column_name]

    if keep_nan:
        # If we want to keep the NaN values use this
        filtered_df = df[column.between(lower, upper) | column.isna()]
    else:
        # If we want to remove the NaN values use this
        filtered_df = df[(column >= lower) & (column <= upper)]

    return filtered_df

df = range_removal(1, 10, 'mood')
df = range_removal(-2, 2, 'circumplex.arousal')
df = range_removal(-2, 2, 'circumplex.valence')
df = range_removal(0, 1, 'activity')

In [97]:
# Remove the negative instances for the applications
def remove_negative_values(columns, df):
    for column in columns:
        # Keep the NaN values and replace the negative values with 0
        df[column] = df.loc[~(df[column] < 0), column]

    return df

df = remove_negative_values(['appCat.builtin', 'appCat.communication', 'appCat.entertainment', 'appCat.finance', 'appCat.game', 'appCat.office', 'appCat.other', 'appCat.social', 'appCat.travel', 'appCat.unknown', 'appCat.utilities', 'appCat.weather', 'screen', 'call', 'sms'], df)


In [90]:
# For each column remove outliers that are more than 3 standard deviations away from the mean
def remove_outliers(df):
    for column in df.columns:
        # Keep the NaN values and replace the outliers with 0
        df[column] = df.loc[~(np.abs(stats.zscore(df[column])) > 3), column]

    return df

df = remove_outliers(df)
df


In [76]:
# remove outliers from the dataframe
def remove_outliers(dataframe):
    for column in dataframe.columns:
        dataframe['z_score'] = np.abs((dataframe[column] - dataframe[column].mean()) / dataframe[column].std())
        outliers = dataframe[dataframe['z_score'] > 3]
        print(outliers)
        dataframe.drop(outliers.index, inplace=True)
        dataframe.drop('z_score', axis=1, inplace=True)

df = remove_outliers(df)
#df


variable                     activity  appCat.builtin  appCat.communication  \
id      date       time                                                       
AS14.01 2014-03-21 14:00:00  0.725000             NaN                   NaN   
                   15:00:00  0.798387             NaN                   NaN   
        2014-03-26 18:00:00  0.822034             NaN                   NaN   
        2014-04-01 11:00:00  0.785124             NaN                   NaN   
        2014-04-26 12:00:00  0.958333             NaN                   NaN   
...                               ...             ...                   ...   
AS14.33 2014-05-13 20:00:00  1.000000             NaN                   NaN   
        2014-05-25 03:00:00  0.697479             NaN                   NaN   
                   04:00:00  0.775000             NaN                   NaN   
                   06:00:00  0.983333             NaN                   NaN   
        2014-05-30 14:00:00  0.731092             Na

KeyError: 'z_score'

In [82]:
# calculate the z-score for each data point
df['z_score'] = np.abs((df['appCat.unknown'] - df['appCat.unknown'].mean()) / df['appCat.unknown'].std())

# identify any data points with a z-score greater than 3 (a common threshold for identifying outliers)
outliers = df[df['z_score'] > 3]
outliers

variable                            activity  appCat.builtin  \
id      date       time                                        
AS14.03 2014-04-10 22:18:19.752000       NaN             NaN   
AS14.07 2014-04-23 23:22:22.224000       NaN             NaN   
AS14.09 2014-04-01 17:35:46.552000       NaN             NaN   
AS14.20 2014-03-20 22:31:20.286000       NaN             NaN   
        2014-03-22 14:19:23.934000       NaN             NaN   
        2014-03-24 21:53:34.308000       NaN             NaN   
        2014-03-30 10:41:11.243000       NaN             NaN   
AS14.23 2014-03-22 20:37:11.877000       NaN             NaN   
                   20:46:31.323000       NaN             NaN   
        2014-03-27 09:48:49.051000       NaN             NaN   
AS14.25 2014-05-06 23:08:16.809000       NaN             NaN   
                   23:17:42.648000       NaN             NaN   
AS14.26 2014-05-18 20:36:10.520000       NaN             NaN   
AS14.30 2014-04-21 09:39:25.503000       NaN             NaN   
        2014-04-22 11:24:09.240000       NaN             NaN   

variable                            appCat.communication  \
id      date       time                                    
AS14.03 2014-04-10 22:18:19.752000                   NaN   
AS14.07 2014-04-23 23:22:22.224000                   NaN   
AS14.09 2014-04-01 17:35:46.552000                   NaN   
AS14.20 2014-03-20 22:31:20.286000                   NaN   
        2014-03-22 14:19:23.934000                   NaN   
        2014-03-24 21:53:34.308000                   NaN   
        2014-03-30 10:41:11.243000                   NaN   
AS14.23 2014-03-22 20:37:11.877000                   NaN   
                   20:46:31.323000                   NaN   
        2014-03-27 09:48:49.051000                   NaN   
AS14.25 2014-05-06 23:08:16.809000                   NaN   
                   23:17:42.648000                   NaN   
AS14.26 2014-05-18 20:36:10.520000                   NaN   
AS14.30 2014-04-21 09:39:25.503000                   NaN   
        2014-04-22 11:24:09.240000                   NaN   

variable                            appCat.entertainment  appCat.finance  \
id      date       time                                                    
AS14.03 2014-04-10 22:18:19.752000                   NaN             NaN   
AS14.07 2014-04-23 23:22:22.224000                   NaN             NaN   
AS14.09 2014-04-01 17:35:46.552000                   NaN             NaN   
AS14.20 2014-03-20 22:31:20.286000                   NaN             NaN   
        2014-03-22 14:19:23.934000                   NaN             NaN   
        2014-03-24 21:53:34.308000                   NaN             NaN   
        2014-03-30 10:41:11.243000                   NaN             NaN   
AS14.23 2014-03-22 20:37:11.877000                   NaN             NaN   
                   20:46:31.323000                   NaN             NaN   
        2014-03-27 09:48:49.051000                   NaN             NaN   
AS14.25 2014-05-06 23:08:16.809000                   NaN             NaN   
                   23:17:42.648000                   NaN             NaN   
AS14.26 2014-05-18 20:36:10.520000                   NaN             NaN   
AS14.30 2014-04-21 09:39:25.503000                   NaN             NaN   
        2014-04-22 11:24:09.240000                   NaN             NaN   

variable                            appCat.game  appCat.office  appCat.other  \
id      date       time                                                        
AS14.03 2014-04-10 22:18:19.752000          NaN            NaN           NaN   
AS14.07 2014-04-23 23:22:22.224000          NaN            NaN           NaN   
AS14.09 2014-04-01 17:35:46.552000          NaN            NaN           NaN   
AS14.20 2014-03-20 22:31:20.286000          NaN            NaN           NaN   
        2014-03-22 14:19:23.934000          NaN            NaN           NaN   
        2014-03-24 21:53:34.308000   

In [93]:
column_name = 'appCat.unknown'

Q1 = df[column_name].quantile(0.25)
Q3 = df[column_name].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

print(lower_bound)

outliers = df[df[column_name] > upper_bound]

#outliers = df[(df[column_name] < lower_bound) | (df[column_name] > upper_bound)]
outliers

-54.10074999999999


variable                            activity  appCat.builtin  \
id      date       time                                        
AS14.01 2014-03-25 19:22:27.391000       NaN             NaN   
AS14.03 2014-04-04 23:29:28.384000       NaN             NaN   
        2014-04-10 22:18:19.752000       NaN             NaN   
        2014-04-20 15:23:17.914000       NaN             NaN   
        2014-05-05 22:30:32.934000       NaN             NaN   
...                                      ...             ...   
AS14.30 2014-04-21 09:39:25.503000       NaN             NaN   
                   22:11:00.549000       NaN             NaN   
        2014-04-22 11:24:09.240000       NaN             NaN   
        2014-04-26 01:25:01.706000       NaN             NaN   
        2014-04-29 20:18:08.099000       NaN             NaN   

variable                            appCat.communication  \
id      date       time                                    
AS14.01 2014-03-25 19:22:27.391000                   NaN   
AS14.03 2014-04-04 23:29:28.384000                   NaN   
        2014-04-10 22:18:19.752000                   NaN   
        2014-04-20 15:23:17.914000                   NaN   
        2014-05-05 22:30:32.934000                   NaN   
...                                                  ...   
AS14.30 2014-04-21 09:39:25.503000                   NaN   
                   22:11:00.549000                   NaN   
        2014-04-22 11:24:09.240000                   NaN   
        2014-04-26 01:25:01.706000                   NaN   
        2014-04-29 20:18:08.099000                   NaN   

variable                            appCat.entertainment  appCat.finance  \
id      date       time                                                    
AS14.01 2014-03-25 19:22:27.391000                   NaN             NaN   
AS14.03 2014-04-04 23:29:28.384000                   NaN             NaN   
        2014-04-10 22:18:19.752000                   NaN             NaN   
        2014-04-20 15:23:17.914000                   NaN             NaN   
        2014-05-05 22:30:32.934000                   NaN             NaN   
...                                                  ...             ...   
AS14.30 2014-04-21 09:39:25.503000                   NaN             NaN   
                   22:11:00.549000                   NaN             NaN   
        2014-04-22 11:24:09.240000                   NaN             NaN   
        2014-04-26 01:25:01.706000                   NaN             NaN   
        2014-04-29 20:18:08.099000                   NaN             NaN   

variable                            appCat.game  appCat.office  appCat.other  \
id      date       time                                                        
AS14.01 2014-03-25 19:22:27.391000          NaN            NaN           NaN   
AS14.03 2014-04-04 23:29:28.384000          NaN            NaN           NaN   
        2014-04-10 22:18:19.752000          NaN            NaN           NaN   
        2014-04-20 15:23:17.914000          NaN            NaN           NaN   
        2014-05-05 22:30:32.934000          NaN            NaN           NaN   
...                                         ...            ...           ...   
AS14.30 2014-04-21 09:39:25.503000          NaN            NaN           NaN   
                   22:11:00.549000          NaN            NaN           NaN   
        2014-04-22 11:24:09.240000          NaN            NaN           NaN   
        2014-04-26 01:25:01.706000          NaN            NaN           NaN   
        2014-04-29 20:18:08.099000          NaN            NaN           NaN   

variable                            appCat.social  appCat.travel  \
id      date       time                                            
AS14.01 2014-03-25 19:22:27.391000            NaN            NaN   
AS14.03 2014-04-04 23:29:28.384000            NaN            NaN   
        2014-04-10 22:18:19.752000            NaN            NaN   
        2014-04-

In [98]:
# Apply different aggregation functions to different variables
agg_dict = {'mood': 'mean', 'circumplex.arousal': 'mean', 'circumplex.valence': 'mean', 'activity': 'mean', 'screen': 'sum', 'call': 'sum', 'sms': 'sum', 'appCat.builtin': 'sum', 'appCat.communication': 'sum', 'appCat.entertainment': 'sum', 'appCat.finance': 'sum', 'appCat.game': 'sum', 'appCat.office': 'sum', 'appCat.other': 'sum', 'appCat.social': 'sum', 'appCat.travel': 'sum', 'appCat.unknown': 'sum', 'appCat.weather': 'sum', 'appCat.utilities': 'sum'}

# Create df where some of the columns values are summed and for some the mean is taken for each day
df = df.groupby(['date', 'id']).agg(agg_dict).reset_index()
df

variable,date,id,mood,circumplex.arousal,circumplex.valence,activity,screen,call,sms,appCat.builtin,...,appCat.entertainment,appCat.finance,appCat.game,appCat.office,appCat.other,appCat.social,appCat.travel,appCat.unknown,appCat.weather,appCat.utilities
0,2014-02-17,AS14.01,NaN,NaN,NaN,NaN,0.000000,2.0,0.0,0.000,...,0.0,0.0,0.0,0.0,0.000,0.000,0.000,0.0,0.0,0.000
1,2014-02-17,AS14.02,NaN,NaN,NaN,NaN,0.000000,4.0,2.0,0.000,...,0.0,0.0,0.0,0.0,0.000,0.000,0.000,0.0,0.0,0.000
2,2014-02-17,AS14.03,NaN,NaN,NaN,NaN,0.000000,0.0,1.0,0.000,...,0.0,0.0,0.0,0.0,0.000,0.000,0.000,0.0,0.0,0.000
3,2014-02-17,AS14.06,NaN,NaN,NaN,NaN,0.000000,5.0,0.0,0.000,...,0.0,0.0,0.0,0.0,0.000,0.000,0.000,0.0,0.0,0.000
4,2014-02-17,AS14.08,NaN,NaN,NaN,NaN,0.000000,0.0,4.0,0.000,...,0.0,0.0,0.0,0.0,0.000,0.000,0.000,0.0,0.0,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1968,2014-06-05,AS14.24,7.0,0.0,0.80,0.152343,2371.588999,11.0,0.0,1749.277,...,0.0,0.0,0.0,0.0,47.360,171.964,225.737,0.0,0.0,0.000
1969,2014-06-06,AS14.24,7.5,0.5,0.25,0.189476,2733.332000,16.0,0.0,945.898,...,0.0,0.0,0.0,0.0,36.414,0.000,0.000,0.0,0.0,2.072
1970,2014-06-07,AS14.24,8.0,-1.0,0.00,0.081893,5530.377999,0.0,2.0,5027.498,...,0.0,0.0,0.0,0.0,6.214,106.850,0.000,0.0,0.0,0.000
1971,2014-06-08,AS14.24,6.5,-1.5,1.00,0.079510,1131.197999,0.0,0.0,351.389,...,0.0,0.0,0.0,0.0,18.069,190.760,0.000,0.0,0.0,0.000


In [100]:
# Function that removes the starting nan values
def remove_starting_nan_until_n_values(column_name, dataframe, n_param):
    '''
    params: 
        colmun_name -> takes the name of the column as a string
        dataframe -> takes the dataframe object
        n_params -> int that takes the number of real float values before the NaN are removed

    This function removes the starting NaN values before an amount of n float values are detected
    Therefore removing all the instances before any real measurements are performed
    '''

    verbose = False

    unique_users = dataframe['id'].unique()
    new_df = pd.DataFrame()

    for user_i in unique_users:
        df_for_user_i = dataframe[dataframe['id'] == user_i]
        count_nan = 0
        count_float = 0
        count_final = 0

        for value in df_for_user_i[column_name]:
            print('-'*50, '\n', value) if verbose else None
            if math.isnan(value):
                print('NAN', value, type(value), isinstance(value, float)) if verbose else None
                count_nan += 1
                count_float = 0
            if isinstance(value, float):
                print("FLOAT",value, type(value), isinstance(value, float)) if verbose else None
                count_float += 1
            if count_float > n_param:
                n = count_nan 
                added_df = df_for_user_i.iloc[n:]
                added_df
                new_df = new_df.append(added_df)
                break

    return new_df
    
df = remove_starting_nan_until_n_values('mood', df, 4)
        
# num_nan_values = df['mood'].isna().sum()
# print(num_nan_values)
# df


In [101]:
# Create a new column that contains the mean of the previous and next value
# NOTE can still change to take mean of previous n and next n values
def mean_between(column_name, dataframe):

    # sort the dataframe by id
    dataframe = dataframe.sort_values(by='date')

    # get the previous and next NaN values
    dataframe['prev_value'] = dataframe.groupby('id')[column_name].apply(lambda x: x.shift(1).bfill())
    dataframe['next_value'] = dataframe.groupby('id')[column_name].apply(lambda x: x.shift(-1).ffill())

    # take the mean of previous and next values
    dataframe['mean_value'] = dataframe[['prev_value', 'next_value']].mean(axis=1)

    for index, row in dataframe.iterrows():
        if pd.isna(row[column_name]):
            dataframe.loc[index, column_name] = row['mean_value']

    dataframe.drop(['mean_value', 'prev_value', 'next_value'], axis=1, inplace=True)

    return dataframe

df = mean_between('mood', df)
df = mean_between('circumplex.arousal', df)
df = mean_between('circumplex.valence', df)
df = mean_between('activity', df)
df

variable,date,id,mood,circumplex.arousal,circumplex.valence,activity,screen,call,sms,appCat.builtin,...,appCat.entertainment,appCat.finance,appCat.game,appCat.office,appCat.other,appCat.social,appCat.travel,appCat.unknown,appCat.weather,appCat.utilities
302,2014-03-04,AS14.17,7.000000,0.500000,1.000000,0.042303,0.000000,5.0,0.0,0.000,...,0.0,0.0,0.0,0.0,0.000,0.000,0.000,0.0,0.0,0.000
319,2014-03-05,AS14.08,6.333333,0.000000,0.333333,0.005556,0.000000,0.0,0.0,0.000,...,0.0,0.0,0.0,0.0,0.000,0.000,0.000,0.0,0.0,0.000
325,2014-03-05,AS14.17,8.000000,0.666667,1.333333,0.042303,0.000000,1.0,1.0,0.000,...,0.0,0.0,0.0,0.0,0.000,0.000,0.000,0.0,0.0,0.000
344,2014-03-06,AS14.17,7.250000,0.500000,0.750000,0.042303,0.000000,1.0,0.0,0.000,...,0.0,0.0,0.0,0.0,0.000,0.000,0.000,0.0,0.0,0.000
338,2014-03-06,AS14.08,7.200000,-0.600000,0.600000,0.005556,0.000000,0.0,13.0,0.000,...,0.0,0.0,0.0,0.0,0.000,0.000,0.000,0.0,0.0,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1968,2014-06-05,AS14.24,7.000000,0.000000,0.800000,0.152343,2371.588999,11.0,0.0,1749.277,...,0.0,0.0,0.0,0.0,47.360,171.964,225.737,0.0,0.0,0.000
1969,2014-06-06,AS14.24,7.500000,0.500000,0.250000,0.189476,2733.332000,16.0,0.0,945.898,...,0.0,0.0,0.0,0.0,36.414,0.000,0.000,0.0,0.0,2.072
1970,2014-06-07,AS14.24,8.000000,-1.000000,0.000000,0.081893,5530.377999,0.0,2.0,5027.498,...,0.0,0.0,0.0,0.0,6.214,106.850,0.000,0.0,0.0,0.000
1971,2014-06-08,AS14.24,6.500000,-1.500000,1.000000,0.079510,1131.197999,0.0,0.0,351.389,...,0.0,0.0,0.0,0.0,18.069,190.760,0.000,0.0,0.0,0.000


In [92]:
dummy_df = df[df['id'] == 'AS14.17']
dummy_df.head(30)

variable,date,id,mood,circumplex.arousal,circumplex.valence,activity,screen,call,sms,appCat.builtin,...,appCat.entertainment,appCat.finance,appCat.game,appCat.office,appCat.other,appCat.social,appCat.travel,appCat.unknown,appCat.weather,appCat.utilities
302,2014-03-04,AS14.17,7.000000,0.500000,1.000000,0.042303,0.000000,5.0,0.0,0.000,...,0.000,0.00,0.0,0.000,0.000,0.000,0.000,0.0,0.0,0.000
325,2014-03-05,AS14.17,8.000000,0.666667,1.333333,0.042303,0.000000,1.0,1.0,0.000,...,0.000,0.00,0.0,0.000,0.000,0.000,0.000,0.0,0.0,0.000
344,2014-03-06,AS14.17,7.250000,0.500000,0.750000,0.042303,0.000000,1.0,0.0,0.000,...,0.000,0.00,0.0,0.000,0.000,0.000,0.000,0.0,0.0,0.000
368,2014-03-07,AS14.17,7.500000,0.750000,1.000000,0.042303,0.000000,0.0,0.0,0.000,...,0.000,0.00,0.0,0.000,0.000,0.000,0.000,0.0,0.0,0.000
387,2014-03-08,AS14.17,7.600000,0.400000,0.800000,0.042303,0.000000,0.0,0.0,0.000,...,0.000,0.00,0.0,0.000,0.000,0.000,0.000,0.0,0.0,0.000
406,2014-03-09,AS14.17,7.200000,0.800000,1.000000,0.042303,0.000000,5.0,0.0,0.000,...,0.000,0.00,0.0,0.000,0.000,0.000,0.000,0.0,0.0,0.000
423,2014-03-10,AS14.17,7.333333,0.666667,0.666667,0.042303,0.000000,0.0,0.0,0.000,...,0.000,0.00,0.0,0.000,0.000,0.000,0.000,0.0,0.0,0.000
443,2014-03-11,AS14.17,6.200000,0.000000,0.400000,0.042303,0.000000,0.0,0.0,0.000,...,0.000,0.00,0.0,0.000,0.000,0.000,0.000,0.0,0.0,0.000
461,2014-03-12,AS14.17,6.400000,0.200000,0.200000,0.042303,0.000000,0.0,0.0,0.000,...,0.000,0.00,0.0,0.000,0.000,0.000,0.000,0.0,0.0,0.000
480,2014-03-13,AS14.17,7.250000,0.750000,0.750000,0.042303,0.000000,0.0,2.0,0.000,...,0.000,0.00,0.0,0.000,0.000,0.000,0.000,0.0,0.0,0.000


In [82]:
# def nan_values_to_zero(column_name, dataframe):
#     # replace NaN values of a column with 0
#     dataframe[column_name] = dataframe[column_name].fillna(0)
#     return dataframe

# dummy_df = nan_values_to_zero('activity', df)
# dummy_df = dummy_df[dummy_df['id'] == 'AS14.17']
# dummy_df.head(30)

variable,date,id,mood,circumplex.arousal,circumplex.valence,activity,screen,call,sms,appCat.builtin,...,appCat.entertainment,appCat.finance,appCat.game,appCat.office,appCat.other,appCat.social,appCat.travel,appCat.unknown,appCat.weather,appCat.utilities
302,2014-03-04,AS14.17,7.000000,0.500000,1.000000,0.000000,0.000000,5.0,0.0,0.000,...,0.0,0.0,0.0,0.0,0.000,0.000,0.000,0.0,0.0,0.000
319,2014-03-05,AS14.08,6.333333,0.000000,0.333333,0.000000,0.000000,0.0,0.0,0.000,...,0.0,0.0,0.0,0.0,0.000,0.000,0.000,0.0,0.0,0.000
325,2014-03-05,AS14.17,8.000000,0.666667,1.333333,0.000000,0.000000,1.0,1.0,0.000,...,0.0,0.0,0.0,0.0,0.000,0.000,0.000,0.0,0.0,0.000
344,2014-03-06,AS14.17,7.250000,0.500000,0.750000,0.000000,0.000000,1.0,0.0,0.000,...,0.0,0.0,0.0,0.0,0.000,0.000,0.000,0.0,0.0,0.000
338,2014-03-06,AS14.08,7.200000,-0.600000,0.600000,0.000000,0.000000,0.0,13.0,0.000,...,0.0,0.0,0.0,0.0,0.000,0.000,0.000,0.0,0.0,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1968,2014-06-05,AS14.24,7.000000,0.000000,0.800000,0.152343,2371.588999,11.0,0.0,1749.277,...,0.0,0.0,0.0,0.0,47.360,171.964,225.737,0.0,0.0,0.000
1969,2014-06-06,AS14.24,7.500000,0.500000,0.250000,0.189476,2733.332000,16.0,0.0,945.898,...,0.0,0.0,0.0,0.0,36.414,0.000,0.000,0.0,0.0,2.072
1970,2014-06-07,AS14.24,8.000000,-1.000000,0.000000,0.081893,5530.377999,0.0,2.0,5027.498,...,0.0,0.0,0.0,0.0,6.214,106.850,0.000,0.0,0.0,0.000
1971,2014-06-08,AS14.24,6.500000,-1.500000,1.000000,0.079510,1131.197999,0.0,0.0,351.389,...,0.0,0.0,0.0,0.0,18.069,190.760,0.000,0.0,0.0,0.000


In [31]:
def MA_on_missing_values(dataframe, column, n):
    '''
    Performs moving averages on a column for each distinct 'id' in the 'mood' column,
    and creates a new column where NaN values are substituted
    '''
    # Group by 'id' column
    groups = dataframe.groupby('id')

    # Perform moving averages for each group
    dataframe['MA'] = groups[column].ewm(span=n).mean().reset_index(0, drop=True)

    for index, row in dataframe.iterrows():
        if pd.isna(row[column]):
            dataframe.loc[index, column] = row['MA']

    dataframe.drop('MA', axis=1, inplace=True)
    
    return dataframe

df = MA_on_missing_values(df, 'mood', 10)
df = MA_on_missing_values(df, 'circumplex.arousal', 10)
df = MA_on_missing_values(df, 'circumplex.valence', 10)
df = MA_on_missing_values(df, 'activity', 10)




variable,date,id,mood,circumplex.arousal,circumplex.valence,activity,screen,call,sms,appCat.builtin,...,appCat.entertainment,appCat.finance,appCat.game,appCat.office,appCat.other,appCat.social,appCat.travel,appCat.unknown,appCat.weather,appCat.utilities
302,2014-03-04,AS14.17,7.000000,0.500000,1.000000,NaN,0.000000,5.0,0.0,0.000,...,0.0,0.0,0.0,0.0,0.000,0.000,0.000,0.0,0.0,0.000
319,2014-03-05,AS14.08,6.333333,0.000000,0.333333,NaN,0.000000,0.0,0.0,0.000,...,0.0,0.0,0.0,0.0,0.000,0.000,0.000,0.0,0.0,0.000
325,2014-03-05,AS14.17,8.000000,0.666667,1.333333,NaN,0.000000,1.0,1.0,0.000,...,0.0,0.0,0.0,0.0,0.000,0.000,0.000,0.0,0.0,0.000
344,2014-03-06,AS14.17,7.250000,0.500000,0.750000,NaN,0.000000,1.0,0.0,0.000,...,0.0,0.0,0.0,0.0,0.000,0.000,0.000,0.0,0.0,0.000
338,2014-03-06,AS14.08,7.200000,-0.600000,0.600000,NaN,0.000000,0.0,13.0,0.000,...,0.0,0.0,0.0,0.0,0.000,0.000,0.000,0.0,0.0,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1968,2014-06-05,AS14.24,7.000000,0.000000,0.800000,0.152343,2371.588999,11.0,0.0,1749.277,...,0.0,0.0,0.0,0.0,47.360,171.964,225.737,0.0,0.0,0.000
1969,2014-06-06,AS14.24,7.500000,0.500000,0.250000,0.189476,2733.332000,16.0,0.0,945.898,...,0.0,0.0,0.0,0.0,36.414,0.000,0.000,0.0,0.0,2.072
1970,2014-06-07,AS14.24,8.000000,-1.000000,0.000000,0.081893,5530.377999,0.0,2.0,5027.498,...,0.0,0.0,0.0,0.0,6.214,106.850,0.000,0.0,0.0,0.000
1971,2014-06-08,AS14.24,6.500000,-1.500000,1.000000,0.079510,1131.197999,0.0,0.0,351.389,...,0.0,0.0,0.0,0.0,18.069,190.760,0.000,0.0,0.0,0.000


In [27]:
def select_imputation_technique(mean_values, dataframe):

    if mean_values == True:
        dataframe = mean_between('mood', dataframe)
        dataframe = mean_between('circumplex.arousal', dataframe)
        dataframe = mean_between('circumplex.valence', dataframe)
        dataframe = mean_between('activity', dataframe)
        dataframe = nan_values_to_zero('activity', dataframe)
    else:
        dataframe = MA_on_missing_values(dataframe, 'mood', 10)
        dataframe = MA_on_missing_values(dataframe, 'circumplex.arousal', 10)
        dataframe = MA_on_missing_values(dataframe, 'circumplex.valence', 10)
        dataframe = MA_on_missing_values(dataframe, 'activity', 10)
        dataframe = nan_values_to_zero('activity', dataframe)

    return df

df = select_imputation_technique(True, df)
df

variable,date,id,mood,circumplex.arousal,circumplex.valence,activity,screen,call,sms,appCat.builtin,...,appCat.entertainment,appCat.finance,appCat.game,appCat.office,appCat.other,appCat.social,appCat.travel,appCat.unknown,appCat.weather,appCat.utilities
645,2014-03-21,AS14.01,6.20,0.2,0.20,0.134050,17978.907000,6.0,0.0,3139.218,...,1007.456,49.544,0.0,172.206,239.751,4508.500,915.445,0.000,0.000,598.754
670,2014-03-22,AS14.01,6.40,0.6,0.40,0.236880,6142.161000,3.0,1.0,731.429,...,93.324,21.076,0.0,0.000,98.143,439.632,37.305,0.000,0.000,117.621
691,2014-03-23,AS14.01,6.80,0.2,0.80,0.142741,6773.832001,0.0,0.0,1286.246,...,94.346,43.403,0.0,0.000,72.823,900.839,0.000,0.000,30.386,30.086
714,2014-03-24,AS14.01,6.00,0.8,0.00,0.078961,15047.351001,10.0,0.0,866.956,...,976.971,34.106,0.0,3.010,66.558,3223.626,419.805,0.000,0.000,178.732
739,2014-03-25,AS14.01,6.75,0.5,0.50,0.098374,21475.354999,0.0,1.0,1032.768,...,68.206,43.054,0.0,0.000,178.819,1919.471,0.000,235.223,0.000,222.893
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1968,2014-06-05,AS14.24,7.00,0.0,0.80,0.152343,2371.588999,11.0,0.0,1749.277,...,0.000,0.000,0.0,0.000,47.360,171.964,225.737,0.000,0.000,0.000
1969,2014-06-06,AS14.24,7.50,0.5,0.25,0.189476,2733.332000,16.0,0.0,945.898,...,0.000,0.000,0.0,0.000,36.414,0.000,0.000,0.000,0.000,2.072
1970,2014-06-07,AS14.24,8.00,-1.0,0.00,0.081893,5530.377999,0.0,2.0,5027.498,...,0.000,0.000,0.0,0.000,6.214,106.850,0.000,0.000,0.000,0.000
1971,2014-06-08,AS14.24,6.50,-1.5,1.00,0.079510,1131.197999,0.0,0.0,351.389,...,0.000,0.000,0.0,0.000,18.069,190.760,0.000,0.000,0.000,0.000


In [102]:
df.to_csv('Datasets/cleaned_data.csv', index=False)